Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [81]:
import numpy as np
import cv2

# Repeatability
np.random.seed(0)

VFILENAME = "walking.mp4" #Video name
HEIGHT = 406 
WIDTH = 722

Load video frames from file

In [82]:
def get_frames(filename):
    # cv.VideoCapture return a video object
    video = cv2.VideoCapture(filename)
    # while the video is opened we take the frames 
    while video.isOpened():
        # read() returns a status flag and the frame (array of pixel color values)
        ret, frame=video.read()
        if ret: 
            # as long as the status valid, we will yeild the frame
            # this will turns the function to a python generator 
            # to call it over and over again and to keep looping and 
            # yeilding frames
            yield frame
        else:
            break
    video.release() # release the resources
    yield None
    return []

Creating a particle cloud

In [83]:
NUM_PARTICLES = 150 # 150 is enough now,
VEL_RANGE=0.5 #Intitial velocity range (assuming the target is not moving faster that 0.5 pixel over a frame)
def initialize_particles():
    # particles array (NUM_PATICLES*4) filled with random numbers between NUM_PATICLES 0 and 1
    particles = np.random.rand(NUM_PARTICLES,4)
    # scaling the numbers :first colomn ->x position, 2nd -> y position , 
    # 3th & 4th -> velocity x,y
    particles = particles * np.array((WIDTH, HEIGHT, VEL_RANGE,VEL_RANGE)  )
    # velocities centered at zero => shifting velocities down
    particles[:,2:4] -= VEL_RANGE/2.0  # all rows with 2nd and 4th colomn
    return []

Moving particles according to their velocity state

In [84]:
# Updating the x,y of the particles according to the velocity
def apply_velocity(particles):
    particles[:][0] += particles[:][2] # x = x + u
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [85]:
def enforce_edges(particles):
    for i in len(particles):
        particles[i,0] = max(0,min(WIDTH-1, particles[i,0])) # prevent x from falling inder zero or out of the upper edge
        particles[i,1] = max(0,min(HEIGHT-1, particles[i,])) # prevent y
    return particles

Measure each particle's quality

In [86]:
def compute_errors(particles, frame):
    errors = np.zeros(NUM_PARTICLES) # color differences from the target
    TARGET_COLOUR = np.array( (189,105,82) ) # Blue top sleeve pixel colour
#    TARGET_COLOUR = np.array( (148, 73, 49) ) # Blue top sleeve pixel colour
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y = int(particles[i,1])
        pixel_colour = frame[ y, x, : ] # pulling the three values of color of the pixel
        errors[i] = np.sum( ( TARGET_COLOUR - pixel_colour )**2 ) # MSE in colour space
    return errors

Assign weights to the particles based on their quality of match

In [87]:
# we want to keep the pixels with small errors bwcuase that means that they are mor similar to the target
# less error -> higher weight
def compute_weights(errors):
    weights = np.max(errors) - errors
    # set the weights of the edges to zero
    weights[ 
        (particles[ :,0 ] == 0) |
        (particles[ :,0 ] == WIDTH-1) |
        (particles[ :,1 ] == 0) |
        (particles[ :,1 ] == HEIGHT-1)
    ] = 0.0
    
    # Make weights more sensitive to colour difference.
    # Cubing a set of numbers in the interval [0,1], 
    #the farther a number is from 1, the more it gets squashed toward zero
    weights = weights**4
    
    return weights

Resample particles according to their weights

In [88]:
def resample(particles, weights):
    # Normalize to get valid PDF, to be used as a probability distribution
    probabilities = weights / np.sum(weights)

    # Resample according to the previous probabilities 
    # Building a new particle array by sampling from the previous array 
    #the values with high weight will be choosen multiple times
    # the vlaues with low weight may not be chosen at all
    indices = np.random.choice(
        NUM_PARTICLES, # sampling from zero to NUM_PARTICLES
        size=NUM_PARTICLES, # number of points to be sampled 
        p=probabilities) # probability distribution
    particles = particles[ indices, : ] # building the particles according to the indices from the sampling
    
    # Take average over all particles, best-guess for location of the target
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    return particles, (int(x),int(y))

Fuzz the particles

In [89]:
def apply_noise(particles):
    # Noise is good!  Noise expresses our uncertainty in the target's position and velocity
    # We add small variations to each hypothesis that were samples from the best ones in last iteration.
    # The target's position and velocity may have changed since the last frame,
    #some of the fuzzed hypotheses will match these changes.
    # We will apply gaussian noise
    POS_SIGMA = 1.0 #standard daviation
    VEL_SIGMA = 0.5
    noise = np.concatenate(
        (
            np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)), # x position
            np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
            np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1)),
            np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1))
        ),
        axis=1 # concatenate colomn wise 
    )
    particles += noise
    return particles

Display the video frames

In [90]:
def display(frame, particles, location):
        if len(particles)>0:
            for i in range(NUM_PARTICLES):
                x=int(particles[i,0]) #we cast the values to int to use them as Pixel coordinates
                y=int(particles[i,1])
                # draw thaat particle as a circle on top of the frame
                # parameters for circle() # frame, circle center, radius, color (BGR not RGB)=green,thickness in pixel
                cv2.circle(frame,(x,y),1,(0,255,0),1) 
        # we use location to draw a circle where the target is 
        if len(location)> 0: #location is a tuple of x and y
            cv2.circle(frame,location,15,(0,0,255),5) # color is red
        cv2.imshow('frame',frame) # display the video frame
        # video playback and stopping it
        if cv2.waitKey(30)== 27: # wait for 30 milisecond for a keypress
            if cv2.waitKey(0)==27: # wait infinite amount of time for a key press
                return True # to stop the playback
        return False

Main routine

In [ ]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break
    # we should include the velocity in the particle state, to keep the circle following the target
    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()
